In [111]:
import utils.crypto_helper as crypto_helper
import utils.google_helper as google_helper
import pandas as pd

## Import Data

In [112]:
coins = crypto_helper.infoCoinMarkets()
coins.head(5)


,Crypto_ID,Crypto_Name,Crypto_Price,Market_Cap,Total_Supply,Circulating_Supply,Price_High_24h,Price_Low_24h,Price_Change_Perchentage_24h
0,1,Bitcoin,17909.980,344619416298,2.100000e+07,1.923362e+07,17942.240,17463.600000,2.55603
1,2,Ethereum,1331.700,160463076760,1.205219e+08,1.205219e+08,1341.760,1291.920000,3.07840
2,3,Tether,1.001,66051963364,6.597237e+10,6.597237e+10,1.003,0.987628,0.17495
3,4,USD Coin,1.000,45201718265,4.516089e+10,4.516581e+10,1.003,0.984953,0.55293
4,5,BNB,273.940,44704766520,1.632770e+08,1.632770e+08,279.740,265.890000,3.02454


In [113]:
coins.dtypes

Crypto_ID                         int64
Crypto_Name                      object
Crypto_Price                    float64
Market_Cap                        int64
Total_Supply                    float64
Circulating_Supply              float64
Price_High_24h                  float64
Price_Low_24h                   float64
Price_Change_Perchentage_24h    float64
dtype: object

## Data Quality

In [110]:
# dublicates 
coins.duplicated()


0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Length: 93, dtype: bool

In [115]:
# missing values
coins.isna().sum()

Crypto_ID                       0
Crypto_Name                     0
Crypto_Price                    0
Market_Cap                      0
Total_Supply                    7
Circulating_Supply              0
Price_High_24h                  0
Price_Low_24h                   0
Price_Change_Perchentage_24h    0
dtype: int64

In [116]:
# replace missing values
coins = coins.dropna()
coins.isna().sum()


Crypto_ID                       0
Crypto_Name                     0
Crypto_Price                    0
Market_Cap                      0
Total_Supply                    0
Circulating_Supply              0
Price_High_24h                  0
Price_Low_24h                   0
Price_Change_Perchentage_24h    0
dtype: int64

## Load Data to BigQuery

In [117]:
google = google_helper.GoogleHelper()
# google.load_data_from_dataframe(table='dim_Cryptocurrency', dataframe=coins)

In [118]:
results = google.query("""
    SELECT Crypto_ID, Crypto_Name, Crypto_Price, Price_High_24h, Price_Low_24h
      FROM {}.{}.{}
    LIMIT 5
""".format(google.get_project_id(), google.get_dataset(), 'dim_Cryptocurrency'))

for el in results:
    print(el['Crypto_ID'], el['Crypto_Name'], el['Crypto_Price'], el['Price_High_24h'], el['Price_Low_24h'])
    

72 Dash 48.01 49.29 47.16
52 Huobi 6.16 6.29 6.1
87 THORChain 1.37 1.4 1.34
19 Uniswap 5.96 6.02 5.79
24 Chainlink 6.69 6.73 6.49


In [119]:
# Check if the High_Price is lower than actual price and Low_Price is higher than actual price

results = google.query("""
    SELECT Crypto_Name, Crypto_Price, Price_High_24h, Price_Low_24h
      FROM {}.{}.{}
    WHERE Price_High_24h < Crypto_Price and Crypto_Price < Price_Low_24h
""".format(google.get_project_id(), google.get_dataset(), 'dim_Cryptocurrency'))

for el in results:
    print(el['Crypto_Name'])